In [1]:
import warmth
from pathlib import Path

In [2]:
maps_dir = Path("./data/")

In [3]:
model = warmth.Model()

In [4]:
inputs = model.builder.input_horizons_template

In [5]:
#Add surface grids to the table. You can use other method as well
inputs.loc[0]=[0,"0.gri",None,"Onlap"]
inputs.loc[1]=[66,"66.gri",None,"Onlap"]
inputs.loc[2]=[100,"100.gri",None,"Onlap"]
inputs.loc[3]=[163,"163.gri",None,"Erosive"]
inputs.loc[4]=[168,"168.gri",None,"Erosive"]
inputs.loc[5]=[170,"170.gri",None,"Onlap"]
inputs.loc[6]=[182,"182.gri",None,"Erosive"]
model.builder.input_horizons=inputs
inputs

,Age,File_name,Facies_maps,Stratigraphy
0,0,0.gri,None,Onlap
1,66,66.gri,None,Onlap
2,100,100.gri,None,Onlap
3,163,163.gri,None,Erosive
4,168,168.gri,None,Erosive
5,170,170.gri,None,Onlap
6,182,182.gri,None,Erosive


In [6]:
inc = 2000
model.builder.define_geometry(maps_dir/"0.gri",xinc=inc,yinc=inc,fformat="irap_binary")

In [7]:
model.builder.extract_nodes(4,maps_dir)

In [8]:
model.parameters.time_start

182

In [9]:
from warmth.data import haq87
model.builder.set_eustatic_sea_level(haq87)

In [10]:
for i in model.builder.iter_node():
    i.rift=[[182,175]]

In [11]:
model.simulator.simulate_every = 1

In [12]:
model.builder.n_valid_node

102

In [14]:
model.simulator.simulate_every=1

In [15]:
%%time
model.simulator.run(save=True,purge=True)

Processing... |################################| 102/102


CPU times: user 471 ms, sys: 224 ms, total: 696 ms
Wall time: 31.6 s


In [16]:
from warmth.mesh_model import run
run(model,start_time=model.parameters.time_start,end_time=0)

Rebuild/reload mesh at tti= 182
Using 1D Node parameters NodeParameters1D(shf=0.03, hc=30000.0, hw=3600.0, hLith=130000.0, kLith=3.109, kCrust=2.5, kAsth=100, rhp=2, crustliquid=2500.0, crustsolid=2800.0, lithliquid=2700.0, lithsolid=3300.0, asthliquid=2700.0, asthsolid=3200.0, T0=5, Tm=1330.0, qbase=0.03)
builing
saving


: 